# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
!pip install gmaps

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
city_data_df = pd.read_csv('clean_city_data.csv')
city_data_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,19.99,73,90,6.91,US,1612072032
1,1,rikitea,-23.1203,-134.9692,79.38,74,39,9.57,PF,1612072032
2,2,kavieng,-2.5744,150.7967,83.57,73,99,11.90,PG,1612072032
3,3,khatanga,71.9667,102.5000,-47.76,78,43,1.52,RU,1612072033
4,4,kabanjahe,3.1001,98.4908,82.40,89,75,5.75,ID,1612072033


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(api_key = g_key)

In [31]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] <80) \
                                    & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,tema,5.6670,-0.0166,78.80,94,0,6.91,GH,1612072054
132,132,lahan,26.7296,86.4951,70.36,30,0,5.73,NP,1612071905
151,151,bikaner,28.0167,73.3000,75.43,15,0,7.61,IN,1612071964
166,166,pokaran,26.9167,71.9167,76.37,14,0,7.92,IN,1612072074
183,183,along,28.1667,94.7667,72.75,28,0,1.19,IN,1612072078
191,191,victoria,22.2855,114.1577,73.40,40,0,9.22,HK,1612072049
317,317,pochutla,15.7432,-96.4661,73.00,86,0,1.01,MX,1612072112
331,331,puerto escondido,15.8500,-97.0667,75.90,64,0,4.07,MX,1612072032
364,364,jati,24.3539,68.2671,77.47,31,0,6.60,PK,1612072123
424,424,calamar,10.2511,-74.9146,77.00,83,0,4.61,CO,1612072137


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng",]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
83,tema,GH,5.6670,-0.0166,
132,lahan,NP,26.7296,86.4951,
151,bikaner,IN,28.0167,73.3000,
166,pokaran,IN,26.9167,71.9167,
183,along,IN,28.1667,94.7667,
191,victoria,HK,22.2855,114.1577,
317,pochutla,MX,15.7432,-96.4661,
331,puerto escondido,MX,15.8500,-97.0667,
364,jati,PK,24.3539,68.2671,
424,calamar,CO,10.2511,-74.9146,


In [42]:
params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key}

for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(url, params=params)
    name_address = name_address.json()
    if i == 8:
        pprint(name_address)
    try:
        hotel_df.loc[i, "Hotel Name"] = name_address ["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing Field... skipping to next")
hotel_df

Missing Field... skipping to next


,City,Country,Lat,Lng,Hotel Name
83,tema,GH,5.6670,-0.0166,Beulah Royale
132,lahan,NP,26.7296,86.4951,Hotel Durbar Pvt. Ltd.
151,bikaner,IN,28.0167,73.3000,The Laxmi Niwas Palace
166,pokaran,IN,26.9167,71.9167,Motel Pokaran - RTDC
183,along,IN,28.1667,94.7667,Hotel West
191,victoria,HK,22.2855,114.1577,Mini Hotel Central
317,pochutla,MX,15.7432,-96.4661,Hotel Posada San Jose
331,puerto escondido,MX,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
364,jati,PK,24.3539,68.2671,
424,calamar,CO,10.2511,-74.9146,Hostal Katyalex


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))